In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算y关于x的梯度之前，需要一个地方来存储梯度

In [2]:
x.requires_grad_(True)
# 等价于 x = torch.arange(4.0, requires_grad=True)

x.grad # 访问导数。初始一般为None

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

grad_fn 意思是torch自动构造了计算图

通过调用反向传播函数 来自动计算y关于x每个分量的梯度

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
# 验证
x.grad == 4*x

tensor([True, True, True, True])

现在 我们要计算x的另一个函数

In [7]:
# 在默认情况下，pytorch会累积梯度，我们需要清除之前的值
x.grad.zero_()

y = x.sum() # x.sum()等价于x与全1向量的内积
y.backward()
x.grad

tensor([1., 1., 1., 1.])

将某些计算移动到记录的计算图之外

用途：将部分网络固定

In [9]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到变量的梯度

In [13]:
def f(a):
    b = a*2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d/a

tensor(True)